# Preamble

In [2]:
from IPython.display import display, Markdown, HTML

# Make connection to Slack

In [3]:
from private_config import SLACK_TOKEN

In [4]:
from slackclient import SlackClient

sc = SlackClient(SLACK_TOKEN)

In [5]:
channel_info = sc.api_call("channels.list")
channels = channel_info['channels']

In [6]:
[(k['name'], k['id']) for k in channels]

[(u'building_comments', u'C6GQ4GWMT'),
 (u'building-comments', u'C6GKKF7U2'),
 (u'cafe_comments', u'C6FCGM656'),
 (u'external_relations', u'C2CMBSDG8'),
 (u'general', u'C2CLS7MHC'),
 (u'movequestions', u'C3WHBE9JN'),
 (u'random', u'C2CM9H9BL'),
 (u'theatreclub', u'C2CLV66QG')]

In [7]:
users = sc.api_call("users.list")
users_map = dict([(x['id'], x['name']) for x in users['members']])

# Get messages of interest

(i.e. building_comments and cafe_comments from actual users)

In [8]:
included_channels = [u'building_comments', u'cafe_comments']
channel_ids = dict([(k['name'], k['id']) for k in channels if k['name'] in included_channels])
channel_ids

{u'building_comments': u'C6GQ4GWMT', u'cafe_comments': u'C6FCGM656'}

In [9]:
channel_comments = {}

for cid in channel_ids:
    print "grabbing history for %s..." % cid
    # grab the history for each
    results = sc.api_call("channels.history", channel=channel_ids[cid], count=1000)
    channel_comments[cid] = results['messages']

grabbing history for building_comments...
grabbing history for cafe_comments...


In [10]:
# this transformed version is used elsewhere
chat_messages = [
    msg for msg in reversed(channel_comments['cafe_comments'])
    if msg['type'] == 'message' and 'subtype' not in msg and 'thread_ts' not in msg
]

# Aside: Emoji Rendering

In [11]:
import json
with open("./node_modules/emoji-datasource-apple/emoji.json") as fp:
    emoji = json.load(fp)
    
# build a mapping from shortname to emoji info that we'll use later on
shortname_to_emoji = dict([(x['short_name'], x) for x in emoji])

# Format and display messages

In [12]:
from itertools import groupby
from datetime import datetime
from pytz import timezone

In [13]:
# nice date formatting
def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def custom_strftime(format, t):
    return t.strftime(format).replace('{S}', str(t.day) + suffix(t.day))

print custom_strftime('%B {S}, %Y', datetime.now())

August 5th, 2017


In [14]:
formatted_chats = [
    {
        'datetime': (datetime
        .fromtimestamp(float(msg['ts']), timezone('UTC'))
        .astimezone(timezone('US/Eastern'))),
        'text': msg['text']
    }
    for msg in chat_messages
]

In [18]:
def escape_markdown(text):
    return text.replace("*", "\*")

In [19]:
["%s: %s" % (custom_strftime("%A, %B {S}, %-I:%M%p", msg['datetime']), msg['text']) for msg in formatted_chats]

[u'Monday, July 31st, 1:55PM: <@U58RD27SN> ',
 u'Monday, July 31st, 6:36PM: Hey ',
 u'Monday, July 31st, 7:02PM: <@U61K5FHQU> <@U58RD27SN> ',
 u'Monday, July 31st, 7:40PM: White bean soup was great',
 u'Monday, July 31st, 7:40PM: Fresh peach was not edible ',
 u'Monday, July 31st, 7:41PM: Double espresso was perfect',
 u'Tuesday, August 1st, 1:09PM: 2nd day in a row of perfect espresso',
 u'Tuesday, August 1st, 1:10PM: Studybot campus-survey',
 u'Tuesday, August 1st, 1:11PM: studybot campus-survey',
 u'Tuesday, August 1st, 2:26PM: Had a sandwich: turkey, mozzarella, all the veggies on multigrain.  It was very good.',
 u'Wednesday, August 2nd, 9:07AM: made to order omelette: (turkey, onions, and peppers): well-seasoned and perfectly cooked. (the _only_ thing is the cafe is still waiting on hot sauce..) but other than that, glowing reviews for breakfast!',
 u'Wednesday, August 2nd, 9:10AM: :+1::skin-tone-5:',
 u'Wednesday, August 2nd, 9:40AM: STARR should look into their credit card proc

In [18]:
# emit channel messages as markdown
import re

find_names_re = re.compile(r'<@([^>]+)>')
find_emoji_re = re.compile(r':(?P<sn>[^:]+):(:(?P<mod>[^:]+):)?')

def replace_name(m):
    try:
        return "**@%s**" % users_map[m.group(1)]
    except KeyError:
        return "*%s*" % m.group(0)
    
def replace_emoji(m):
    try:
        emoji = shortname_to_emoji[m.group('sn')]
        
        if m.group('mod') and 'skin_variations' in emoji:
            # just print the emoji + variation out as two separate characters
            mod = shortname_to_emoji[m.group('mod')]
            # variation = emoji['skin_variations'][mod['unified']]
            return "&#x%s;&#x%s;" % (emoji['unified'], mod['unified'])
        else:
            return "&#x%s;" % emoji['unified']
    except KeyError:
        return m.group(0)

In [24]:
# compute bounds
start_time = min(min(float(msg['ts']) for msg in messages) for messages in channel_comments.values())
end_time = max(max(float(msg['ts']) for msg in messages) for messages in channel_comments.values())

def localize_ts(ts, tzone='US/Eastern'):
    return (datetime
            .fromtimestamp(float(ts), timezone('UTC'))
            .astimezone(timezone(tzone)))

start_datetime, end_datetime = localize_ts(start_time), localize_ts(end_time)
start_datestr, end_datestr = custom_strftime("%A, %B {S}, %Y", start_datetime), custom_strftime("%A, %B {S}, %Y", end_datetime)

print "%s to %s" % (start_datestr, end_datestr)

Monday, July 31st, 2017 to Friday, August 4th, 2017


In [70]:
# html version

def replace_name_html(m):
    try:
        return '<b style="color: #3393cc;">@%s</b>' % users_map[m.group(1)]
    except KeyError:
        return "<b>%s</b>" % m.group(0)

# gathers output for emission
emissions = []
def emit(mkdown):
    global emissions
    emissions.append(mkdown)

emit(HTML("""<div style="text-align: center; margin-bottom: 2em;">
<h1 style="margin-bottom: 0.5em; font-size: xx-large;">Cornell Tech Weekly Digest</h1>
<div style="font-size: large; line-height: 150%%; color: #777;">messages from %(channel_list)s<br />from %(start_datestr)s to %(end_datestr)s</div>
</div>""" % {
    'channel_list': ", ".join([('<b style="color: black;">#%s</b>' % name) for name in channel_comments]),
    'start_datestr': '<b style="color: black;">%s</b>' % start_datestr,
    'end_datestr': '<b style="color: black;">%s</b>' % end_datestr
}))

# actually iterate over the messages now
for channel_name, comments in channel_comments.items():
    personal_messages = [
        msg for msg in reversed(comments)
        # if msg['type'] == 'message' and 'subtype' not in msg
        if msg['type'] == 'message' and 'subtype' not in msg and 'thread_ts' not in msg
    ]
    localized_chats = [
        {
            'datetime': (datetime
            .fromtimestamp(float(msg['ts']), timezone('UTC'))
            .astimezone(timezone('US/Eastern'))),
            'text': msg['text'],
            'user': msg['user']
        }
        for msg in personal_messages
    ]
    
    # actually print stuff now
    
    emit(HTML('<div style="margin-left: auto; padding: 20px; padding-top: 10px; margin-right: auto; margin-bottom: 1em; max-width: 600px; border-radius: 3px; border: solid 1px #ccc;">'))
    
    emit(HTML('<h2 style="font-size: x-large;">#%s</h2><hr color="#ccc" size="1" style="color: #ccc;" />' % channel_name))

    for day, messages in groupby(localized_chats, lambda x: custom_strftime("%A, %B {S}", x['datetime'])):
        emit(HTML('<h3 style="color: #555;">%s</h3>' % day))
        emit(HTML('<ul style="margin-bottom: 0.5em;">'))
        
        for message in messages:
            final_text = find_names_re.sub(replace_name_html, message['text'])
            final_text = find_emoji_re.sub(replace_emoji, final_text)
            
            data = {
                'text': final_text,
                'timeofday': custom_strftime("%-I:%M %p", message['datetime']),
                'author': users_map[message['user']]
            }
            emit(HTML("""<li style="margin-bottom: 0.5em;"><b style="font-size: larger;">%(author)s</b> <span style="color: #777; font-size: smaller;">%(timeofday)s</span><br />%(text)s</li>""" % data))
        
        emit(HTML("</ul>"))
        
    emit(HTML("</div>"))
        
# ...and final processing/display

display(HTML("".join([em.data for em in emissions])))

# Email result

In [22]:
import yagmail

In [74]:
# ensure that we're still connected
yag = yagmail.SMTP('falquaddoomi')

from markdown import markdown
html = ""

for em in emissions:
    if type(em) == Markdown:
        html += markdown(em.data, safe_mode='escape')
    else:
        html += em.data

# html = "".join([markdown(x.data, safe_mode='escape') for x in emissions])
html = find_emoji_re.sub(replace_emoji, html)

yag.send('falquaddoomi@gmail.com', 'Cornell Tech Weekly Digest (rev2)', html)

{}